# --- Day 8: Memory Maneuver ---

In [1]:
with open('input/day8.txt') as f:
    header = f.readline().strip()

In [2]:
import re
header = [int(v) for v in re.findall(r'([0-9]+)', header)]

In [3]:
test = [int(v) for v in re.findall(r'([0-9]+)','2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2')]

In [52]:
test

[2, 3, 0, 3, 10, 11, 12, 1, 1, 0, 1, 99, 2, 1, 1, 2]

In [4]:
def fill(header, meta=0, nm=0):
    if len(header) == nm:
        for h in header:
            meta += h
    else:
        nc = header[0]
        if nc == 0:
            for i in range(header[1]):
                meta += header[2+i]
            return fill(header[2+header[1]:], meta, nm)
        else:
            nm += header[1]
            for c in range(nc):
                return fill(header[2:], meta, nm)
    return meta

In [5]:
fill(test) == 138

True

In [6]:
fill(header)

RuntimeError: maximum recursion depth exceeded

The header is too long for recursion...

In [62]:
from collections import Counter, defaultdict

In [60]:
# keep records of nodes
ncnode = Counter() # number of children in a node
nmnode = Counter() # number of meta in a node
parent = defaultdict() # parent of each node

# some counters
node = 0 # node counter
nxtnode = 1 #
meta = 0 # sum of meta values

# meaning of next value
nxt = 'nc' # number of children (nc), number of meta values (nm) or a meta value (meta)

for h in header:
    if nxt == 'nc':
        parent[nxtnode] = node
        node = nxtnode
        nxtnode += 1
        ncnode[node] = h
        nxt = 'nm'
    elif nxt == 'nm':
        nmnode[node] = h
        if ncnode[node] > 0:
            gen += 1
            nxt = 'nc'
        elif ncnode[node] == 0:
            nxt = 'meta'
        else: 
            print('dont know...')
    elif nxt == 'meta':
        if nmnode[node] > 0:
            meta += h
            
            nmnode[node] -= 1
        elif nmnode[node] == 0:
            runwhile = True
            while runwhile:
                node = parent[node]
                ncnode[node] -= 1
                if ncnode[node] > 0:
                    parent[nxtnode] = node
                    node = nxtnode
                    nxtnode += 1
                    nxt = 'nm'
                    ncnode[node] = h
                    runwhile = False
                elif (ncnode[node] == 0) & (nmnode[node] > 0):
                    nxt = 'meta'
                    meta += h
                    nmnode[node] -= 1
                    runwhile = False
                else:
                    runwhile = True
        else:
            print('dont know what to do')
    else:
        print('last resort')
            

In [61]:
print('sum of all meta values: {}'.format(meta))

sum of all meta values: 46578


## --- Part Two ---

In [218]:
# keep records of nodes
ncnode = Counter() # number of children in a node
children = Counter() # keep record of number of children
nmnode = Counter() # number of meta in a node
parent = defaultdict() # parent of each node
meta = Counter() # sum of meta values for each node
metalst = defaultdict(list) # list of meta values

# some counters
node = 0 # node counter, note that the first generation = children[1]
nxtnode = 1 #

# meaning of next value
nxt = 'nc' # number of children (nc), number of meta values (nm) or a meta value (meta)

for h in header:
    if nxt == 'nc':
        parent[nxtnode] = node
        node = nxtnode
        nxtnode += 1
        ncnode[node] = h
        children[node] = h
        nxt = 'nm'
    elif nxt == 'nm':
        nmnode[node] = h
        if ncnode[node] > 0:
            gen += 1
            nxt = 'nc'
        elif ncnode[node] == 0:
            nxt = 'meta'
        else: 
            print('dont know...')
    elif nxt == 'meta':
        if nmnode[node] > 0:
            meta[node] += h
            metalst[node].append(h)
            nmnode[node] -= 1
        elif nmnode[node] == 0:
            runwhile = True
            while runwhile:
                node = parent[node]
                ncnode[node] -= 1
                if ncnode[node] > 0:
                    parent[nxtnode] = node
                    node = nxtnode
                    nxtnode += 1
                    nxt = 'nm'
                    ncnode[node] = h
                    children[node] = h
                    runwhile = False
                elif (ncnode[node] == 0) & (nmnode[node] > 0):
                    nxt = 'meta'
                    meta[node] += h
                    metalst[node].append(h)
                    nmnode[node] -= 1
                    runwhile = False
                else:
                    runwhile = True
        else:
            print('dont know what to do')
    else:
        print('last resort')     

In [219]:
sum(meta.values())

46578

In [233]:
parents = defaultdict(list) # lists all children for each parent
for node in parent:
    parents[parent[node]].append(node)

childrentosum = defaultdict(list) # lists the children's meta values that should be summed
for p in parents:
    chlds = parents[p]
    for m in metalst[p]:
        if m <= len(chlds):
            childrentosum[p].append(chlds[m-1])

In [235]:
newmeta = Counter()
done = {1: False}
while not done[1]:
    for node in children:
        if not children[node]:
            newmeta[node] = meta[node]
            done[node] = True
        else:
            if [c for c in childrentosum[node] if c in newmeta.keys()] == childrentosum[node]:
                done[node] = True
                newmeta[node] = sum([newmeta[c] for c in childrentosum[node]])
            else:
                done[node] = False
                pass

In [237]:
print('value of the root node = {}'.format(newmeta[1]))

value of the root node = 31251
